In [ ]:

!pip install datasets transformers scikit-learn
!pip install transformers


import datasets
from datasets import load_dataset
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from transformers import BertTokenizer, BertModel
import torch


ds = load_dataset("google-research-datasets/google_wellformed_query")


df_train = ds["train"].to_pandas()
df_test = ds["test"].to_pandas()


tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
def tokenize_content(content):
    tokens = tokenizer(content, padding='max_length', max_length=50, truncation=True, return_tensors="pt")
    return tokens['input_ids'], tokens['attention_mask']

df_train['input_ids'], df_train['attention_mask'] = zip(*df_train['content'].apply(tokenize_content))
df_test['input_ids'], df_test['attention_mask'] = zip(*df_test['content'].apply(tokenize_content))


model = BertModel.from_pretrained('bert-base-uncased')

def get_embeddings(input_ids, attention_mask):
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
    return outputs.last_hidden_state.mean(dim=1).numpy()

df_train['content_embeddings'] = df_train.apply(lambda x: get_embeddings(x['input_ids'], x['attention_mask']), axis=1)
df_test['content_embeddings'] = df_test.apply(lambda x: get_embeddings(x['input_ids'], x['attention_mask']), axis=1)


X_train = np.vstack(df_train['content_embeddings'].values)
y_train = df_train['rating'].values
X_test = np.vstack(df_test['content_embeddings'].values)
y_test = df_test['rating'].values


X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


rf_model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
rf_model.fit(X_train, y_train)


y_pred_val = rf_model.predict(X_val)


mse_val = mean_squared_error(y_val, y_pred_val)
r2_val = r2_score(y_val, y_pred_val)

print("Validation Mean Squared Error:", mse_val)
print("Validation R² Score:", r2_val)


y_pred_test = rf_model.predict(X_test)
mse_test = mean_squared_error(y_test, y_pred_test)
r2_test = r2_score(y_test, y_pred_test)

print("Test Mean Squared Error:", mse_test)
print("Test R² Score:", r2_test)




INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.1 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


google_wellformed_query.py:   0%|          | 0.00/3.24k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.40k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/17500 [00:00<?, ? examples/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Validation Mean Squared Error: 0.08900504083376946
Validation R² Score: 0.3803377056970708
Test Mean Squared Error: 0.08982807344535071
Test R² Score: 0.37925099170352894
